# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = "Weather.csv"
weather_df = pd.read_csv(weather)
weather_df.head()

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Rikitea,PF,1,1587176746,63,-23.12,-134.97,77.36,1.99
1,Grants Pass,US,1,1587176780,20,42.44,-123.33,77.00,9.17
2,Ushuaia,AR,40,1587176866,61,-54.80,-68.30,48.20,8.05
3,Punta Arenas,CL,14,1587176900,75,-53.15,-70.92,44.60,10.29
4,Port Elizabeth,ZA,20,1587176900,87,-33.92,25.57,60.80,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"].astype(float)
max_intensity = weather_df["Humidity"].max()

In [4]:
fig = gmaps.figure(center=(25,0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=max_intensity)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
fit_df = weather_df.loc[(weather_df["Max Temp"] <80) & (weather_df["Wind Speed"]<2) &
                        (weather_df["Cloudiness"]<10)]
fit_df.dropna
fit_df.count()

City          4
Country       4
Cloudiness    4
Date          4
Humidity      4
Latitude      4
Longitude     4
Max Temp      4
Wind Speed    4
dtype: int64

In [6]:
hotel_df = fit_df.copy()
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Rikitea,PF,1,1587176746,63,-23.12,-134.97,77.36,1.99,
1,Tiznit Province,MA,0,1587176911,66,29.58,-9.50,55.06,0.49,
2,Diego de Almagro,CL,0,1587176923,46,-26.37,-70.05,58.35,0.43,
3,Tailai,CN,0,1587176942,37,46.38,123.40,64.99,1.99,


In [9]:
latitude = hotel_df["Latitude"]
longitude = hotel_df["Longitude"]
length_i = len(hotel_df)

In [10]:
for i in range(length_i):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude[i]},{longitude[i]}",
        "radius": 5000,
        "types": "lodging",
        "keyword": "Hotel",
        "key": g_key
    }
    response = requests.get(base_url, params=params).json()
    try: 
        results = response["results"][i]["name"]
        print(f"Nearest hotel: {results}")
        hotel_df.loc[hotel_df["City"] == hotel_df["City"][i], "Hotel Name"] = results
    except IndexError:
        print("No options available")

Nearest hotel: Pension Maro'i
No options available
Nearest hotel: Hotel Rocas del Norte
Nearest hotel: Baihe Express Hotel


In [16]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
hotel_df = hotel_df.reset_index()
del hotel_df["index"]
hotel_df

,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Rikitea,PF,1,1587176746,63,-23.12,-134.97,77.36,1.99,Pension Maro'i
1,Diego de Almagro,CL,0,1587176923,46,-26.37,-70.05,58.35,0.43,Hotel Rocas del Norte
2,Tailai,CN,0,1587176942,37,46.38,123.40,64.99,1.99,Baihe Express Hotel


In [17]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))